Part A: Classification using SVM (Basic Model)

First, we'll load the dataset, split it into training and testing sets, and standardize the features. Then, we'll train a basic SVM classifier with default parameters.

In [ ]:
# 1. Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# 2. Data Preparation [cite: 23]
# Load the dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Split data into training (70%) and testing (30%) sets 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Standardize features 
# Scaling is important for SVM as it is sensitive to feature ranges
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# 3. Model Training (Basic) [cite: 27]
# Train a basic SVM classifier with default parameters
basic_svm = SVC(random_state=42)
basic_svm.fit(X_train_scaled, y_train)

# 4. Evaluation 
y_pred_basic = basic_svm.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_basic)
cm = confusion_matrix(y_test, y_pred_basic)

print("--- Basic SVM Model Evaluation ---")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(cm)

# Visualize the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=cancer.target_names, yticklabels=cancer.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Basic SVM')
plt.show()